## Labarotorio 4 Inteligencia Artificial 

### Regresion Lineal Polinomica


#### Leer archvivo CSV

In [17]:
#Importar librerias
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### Ajustar Modelo Polinomial

Importar los datos y almacenarlos en una matriz de NumPy.

In [6]:
import numpy as np

data = np.genfromtxt('kc_house_data.csv', delimiter=',', skip_header=1)

Separar los datos de la variable objetivo (price) y la variable predictora (sqft living).

In [7]:
y = data[:, 2]  # price
X = data[:, 5]  # sqft living

Agregar una columna de unos a la matriz de la variable predictora para representar el término de intercepción.

In [8]:
X = np.vstack([X, np.ones(len(X))]).T

Utilizar la fórmula matricial de mínimos cuadrados para ajustar el modelo lineal.

In [9]:
theta = np.linalg.inv(X.T @ X) @ X.T @ y

Utilizar el modelo ajustado para hacer predicciones

In [10]:
new_X = np.array([1500, 2000])  # ejemplos de nuevos valores de sqft living
new_X = np.vstack([new_X, np.ones(len(new_X))]).T
predictions = new_X @ theta

print(predictions)

[377354.6087517  517666.39270042]


### Implementacion Vectorial

Cargar los datos desde el archivo CSV

In [44]:
import numpy as np
import csv

def load_data(file_path, delimiter=","):
    data = []
    with open(file_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=delimiter)
        for row in csv_reader:
            data.append(row)
    return data

Convertir los datos a una matriz de numpy

In [57]:
def data_to_matrix(data, target_col, feature_cols):
    data = np.array(data[1:], dtype=float)[:, 1:] # Omitir la primera columna y la fila de encabezados
    X = data[:, feature_cols]
    y = data[:, target_col]
    return X, y

Normalizar los datos

In [46]:
def normalize(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    X_norm = (X - mean) / std
    return X_norm, mean, std

Calcular el costo (error) del modelo

In [47]:
def compute_cost(X, y, theta):
    m = len(y)
    h = X.dot(theta)
    cost = np.sum((h - y) ** 2) / (2 * m)
    return cost

Implementar el descenso de gradiente

In [48]:
def gradient_descent(X, y, theta, alpha, num_iters):
    m = len(y)
    J_history = np.zeros(num_iters)

    for i in range(num_iters):
        h = X.dot(theta)
        error = h - y
        grad = X.T.dot(error) / m
        theta -= alpha * grad
        J_history[i] = compute_cost(X, y, theta)

    return theta, J_history

Implementacion

In [58]:
# Cargar los datos desde el archivo
data = load_data("kc_house_data.csv")

# Definir las columnas que se utilizarán como características (features) y etiquetas (target)
feature_cols = [5] # sqft_living
target_col = [2] # price

# Convertir los datos a una matriz de numpy
X, y = data_to_matrix(data, target_col, feature_cols)

# Normalizar los datos
X, mean, std = normalize(X)

# Añadir una columna de unos a la matriz X para el término de sesgo (bias)
X = np.hstack((np.ones((len(y), 1)), X))

# Inicializar los pesos aleatoriamente
theta = np.random.rand(X.shape[1])

# Definir los hiperparámetros del algoritmo de gradiente descendente
alpha = 0.1
num_iters = 1000

# Ejecutar el algoritmo de gradiente descendente
theta, J_history = gradient_descent(X, y, theta, alpha, num_iters)

# Imprimir los pesos del modelo
print("Pesos del modelo:", theta)


ValueError: could not convert string to float: '20141013T000000'

Graficar

In [32]:
import matplotlib.pyplot as plt

plt.plot(range(num_iters), J_history)
plt.xlabel("Iteraciones")
plt.ylabel("Costo (error)")
plt.show()

NameError: name 'num_iters' is not defined

#### Uso de Cross Validation

In [64]:
import numpy as np

def cross_validate(X, y, k=5):
    """
    Divide los datos en k partes y realiza k iteraciones del cross-validation.
    En cada iteración, una de las partes se usa como conjunto de prueba y el resto
    como conjunto de entrenamiento. Retorna los errores de entrenamiento y prueba
    para cada iteración.
    """
    n = X.shape[0]
    indices = np.arange(n)
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]
    split_size = n // k
    train_errors = np.zeros(k)
    test_errors = np.zeros(k)
    for i in range(k):
        test_indices = np.arange(i*split_size, (i+1)*split_size)
        train_indices = np.concatenate((np.arange(0, i*split_size), np.arange((i+1)*split_size, n)))
        X_train = X[train_indices]
        y_train = y[train_indices]
        X_test = X[test_indices]
        y_test = y[test_indices]
        # Entrenar el modelo con los datos de entrenamiento
        weights = fit_polynomial_regression(X_train, y_train)
        # Calcular los errores de entrenamiento y prueba
        train_errors[i] = calculate_error(X_train, y_train, weights)
        test_errors[i] = calculate_error(X_test, y_test, weights)
    return train_errors, test_errors

def fit_polynomial_regression(X, y, degree=1):
    """
    Entrena un modelo de regresión polinomial de grado "degree" utilizando
    los datos de entrada X y los valores objetivo y. Retorna los pesos
    (coeficientes) de la regresión.
    """
    X_poly = polynomial_features(X, degree)
    X_poly_T = np.transpose(X_poly)
    weights = np.linalg.inv(X_poly_T @ X_poly) @ (X_poly_T @ y)
    return weights

def polynomial_features(X, degree=1):
    """
    Expande los datos de entrada X para incluir todas las potencias de grado
    hasta el grado "degree". Retorna una nueva matriz con las características
    expandidas.
    """
    X_poly = X.copy()
    for i in range(2, degree+1):
        X_poly = np.column_stack((X_poly, np.power(X, i)))
    return X_poly

def calculate_error(X, y, weights):
    """
    Calcula el error cuadrático medio para un modelo de regresión y los datos
    de entrada y los valores objetivo. Retorna el error.
    """
    y_pred = predict(X, weights)
    error = np.mean((y - y_pred)**2)
    return error

def predict(X, weights):
    """
    Calcula las predicciones de un modelo de regresión para los datos de entrada
    X utilizando los pesos (coeficientes) dados. Retorna las predicciones.
    """
    return X @ weights

def split_data(X, y, train_ratio=0.8):
    """
    Divide los datos de entrada X y los valores objetivo y en conjuntos de entrenamiento
    y prueba, utilizando el ratio especificado. Retorna los conjuntos de entrenamiento y
    prueba.
    """
    n = X.shape[0]
    train_size = int(train_ratio * n)
    indices = np.arange(n)
    np.random.shuffle(indices)
    train_indices = indices[:train_size]
    test_indices = indices[train_size:]
    X_train = X[train_indices]
    y_train = y[train_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]
    return X_train, y_train, X_test, y_test

# Cargar los datos
data = np.genfromtxt('kc_house_data.csv', delimiter=',', skip_header=True)
feature_cols = [0]
target_col = [1]

# Convertir los datos a una matriz de numpy
X, y = data_to_matrix(data, target_col, feature_cols)

# Normalizar los datos
X, mean, std = normalize(X)

# Dividir los datos en entrenamiento y prueba
X_train, y_train, X_test, y_test = split_data(X, y, train_ratio=0.8)

# Realizar cross-validation con los datos de entrenamiento
train_errors, test_errors = cross_validate(X_train, y_train, k=5)

# Calcular los errores promedio de entrenamiento y prueba
mean_train_error = np.mean(train_errors)
mean_test_error = np.mean(test_errors)

print("Error de entrenamiento promedio: ", mean_train_error)
print("Error de prueba promedio: ", mean_test_error)




Error de entrenamiento promedio:  nan
Error de prueba promedio:  nan
